库准备

In [1]:
import pymysql
from create_db import db_config
connection = pymysql.connect(**db_config)
if connection.open:
    print("数据库连接成功！")

数据库连接成功！


In [2]:
# 获取pid为1的文章下的所有评论，并列出全部信息
import json
pid = 1

with connection.cursor(cursor=pymysql.cursors.DictCursor) as cursor:
    # 获取评论点赞者
    def get_comment_likers(cid):
        cursor.execute(
            """
            WITH likers AS (SELECT * FROM like_comment WHERE like_comment.pid = %s)
            SELECT u.username
            FROM likers AS l
            JOIN users AS u ON l.uid = u.uid
            WHERE l.cid = %s
        """,
            (pid, cid),
        )
        return [row["username"] for row in cursor.fetchall()]

    # 获取评论详细信息
    def get_comments(parent_cid=None):
        base_query = """
            SELECT c.*, u.username, 
            ru.username AS reply_to_user, rc.content AS reply_to_content
            FROM comments c
            LEFT JOIN users u ON c.from_user = u.uid
            LEFT JOIN comments rc ON c.to_comment = rc.cid AND c.pid = rc.pid
            LEFT JOIN users ru ON rc.from_user = ru.uid
            WHERE c.pid = %s
        """
        params = [pid]

        if parent_cid is None:
            base_query += " AND c.master_comment IS NULL"
        else:
            base_query += " AND c.master_comment = %s"
            params.append(parent_cid)

        cursor.execute(base_query, tuple(params))

        comments = []
        for comment in cursor.fetchall():
            comment_dict = {
                "cid": comment["cid"],
                "username": comment["username"],
                "content": comment["content"],
                "reply_to": (
                    {"user": comment['reply_to_user'], "content": comment['reply_to_content']} if comment['reply_to_user'] else None
                ),
                "replies": get_comments(comment["cid"]) if parent_cid is None else None,
                "likers": get_comment_likers(comment["cid"]),
            }
            comments.append(comment_dict)
        return comments

    comments_tree = get_comments()
    json_data = json.dumps(comments_tree)
    print(json_data)

[{"cid": 1, "username": "bob", "content": "Very insightful article on AI!", "reply_to": null, "replies": [{"cid": 4, "username": "eve", "content": "I agree, especially in healthcare applications.", "reply_to": {"user": "bob", "content": "Very insightful article on AI!"}, "replies": null, "likers": ["david"]}, {"cid": 5, "username": "alice", "content": "Also in autonomous vehicles!", "reply_to": {"user": "bob", "content": "Very insightful article on AI!"}, "replies": null, "likers": ["eve"]}, {"cid": 6, "username": "bob", "content": "Great point about ethics. We need more regulation.", "reply_to": {"user": "david", "content": "What about the ethical implications?"}, "replies": null, "likers": ["bob"]}], "likers": ["alice", "charlie", "eve"]}, {"cid": 2, "username": "charlie", "content": "I think AI will change everything.", "reply_to": null, "replies": [], "likers": ["bob", "david"]}, {"cid": 3, "username": "david", "content": "What about the ethical implications?", "reply_to": null, "r

In [3]:
# 注册-增
# *登录*-查
# 修改密码-改

# （当前用户）列出全部公众号-查
# 关注-增
# 取消关注-删

# （当前用户）显示自己所关注的公众号的全部推文-查

# （当前用户）评论某推文-增
# （当前用户）评论某评论-增

# （当前用户）点赞某推文或者评论-增
# （当前用户）取消点赞某推文或者评论-删

In [4]:
#注册
# to be done
# 从前端获取用户名和注册密码信息
name = 'new_user_2'
password = '123'

#哈希（to be done）

#
with connection.cursor(cursor=pymysql.cursors.DictCursor) as cursor:
    #判断是否已经注册过
    sql_has_registered = """
    SELECT users.uid
    FROM users
    WHERE users.username = %s
    """
    cursor.execute(sql_has_registered,(name),)
    
    result = cursor.fetchone()
    if result is None:
        #注册新账号
        sql_register = """
        INSERT INTO users (username, password) 
        VALUES (%s, %s);
        """
        cursor.execute(sql_register,(name,password),)
    else:
        #to be done
        #如何与前端交互
        print('已经注册过')
    
    


In [5]:
#登录
# to be done
# 从前端获取用户名和登录密码信息
name = 'new_user_2'
password = '123'

#反哈希（to be done）

#
with connection.cursor(cursor=pymysql.cursors.DictCursor) as cursor:
    #判断密码是否符合
    sql_login = """
    SELECT users.uid
    FROM users
    WHERE users.username = %s AND users.password = %s
    """
    cursor.execute(sql_login,(name,password),)
    
    result = cursor.fetchall()
    if len(result) == 0:
        #to be done
        #如何与前端交互
        print('用户名或密码错误')
    else:
        #to be done
        #如何与前端交互
        print('登录成功')


登录成功


In [6]:
#修改密码

# to be done
# 从前端获取用户名和修改密码信息
name = 'new_user_1'
new_password = '1243'

# 调试状态
debugg = 1

#反哈希（to be done）

#
with connection.cursor(cursor=pymysql.cursors.DictCursor) as cursor:
    #判断密码是否符合
    sql_change_password = """
    UPDATE users
    SET users.password = %s
    WHERE users.username = %s;
    """
    cursor.execute(sql_change_password,(new_password,name),)
    
    #to be done（通知前端）
    
    #debug
    if debugg == 1:
        #检测是否修改成功
        cursor.execute("""
                    SELECT *
                    FROM users
                    WHERE users.username = %s
                    """,(name))
        result = cursor.fetchall()
        for row in result:
            print(row)

In [7]:
# （当前用户）列出全部公众号-查

#channels
#               nid INT AUTO_INCREMENT PRIMARY KEY,
#               name VARCHAR(50) UNIQUE NOT NULL,
#               created_by INT NOT NULL,
#               created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
#               FOREIGN KEY (created_by) REFERENCES users(uid)

with connection.cursor(cursor=pymysql.cursors.DictCursor) as cursor:
    sql_list_all_channels = """
                    SELECT channels.name
                    FROM channels
                    """
    cursor.execute(sql_list_all_channels)
    result = cursor.fetchall()
    for row in result:
        print(row)
        
# to be done
# 前端需要返回什么

{'name': 'GadgetReviews'}
{'name': 'HealthTips'}
{'name': 'ScienceDaily'}
{'name': 'TechNews'}
{'name': 'TravelBlog'}


In [8]:
# 关注-增

# to be done
# 从前端获取哪个用户关注了哪个公众号信息
user_name = 'new_user_1'
channel_name = 'ScienceDaily'

#要插入subscribe表的内容
user_id = 0
channel_id = 0

# 调试状态
debugg = 1

with connection.cursor(cursor=pymysql.cursors.DictCursor) as cursor:
    #获取uid
    sql_get_user_id_from_username = """
    SELECT users.uid
    FROM users
    WHERE users.username = %s
    """
    cursor.execute(sql_get_user_id_from_username,(user_name),)
    result = cursor.fetchone()
    user_id = result['uid']
    
    #获取nid
    sql_get_channel_id_from_channel_name = """
    SELECT channels.nid
    FROM channels
    WHERE channels.name = %s
    """
    cursor.execute(sql_get_channel_id_from_channel_name,(channel_name))
    result = cursor.fetchone()
    channel_id = result['nid']
    
    #将获得的(uid,nid)插入subscribe表中,如果已经关注,忽略
    sql_has_already_subscribed = """
    SELECT *
    FROM subscribe AS s
    WHERE s.uid = %s AND s.nid = %s
    """
    cursor.execute(sql_has_already_subscribed,(user_id,channel_id))
    result = cursor.fetchall()
    if len(result) == 0:
        sql_subscribe = """
            INSERT INTO subscribe (uid, nid)
            VALUES(%s,%s)
        """
        cursor.execute(sql_subscribe,(user_id,channel_id))
    
    # to be done
    ##### 通知前端关注成功
    
    #####
    
    #debug
    if debugg:
        sql_debug = """
            SELECT *
            FROM subscribe AS s
            WHERE s.uid = %s AND s.nid = %s
        """
        cursor.execute(sql_debug,(user_id,channel_id))
        result = cursor.fetchall()
        for row in result:
            print(row)


TypeError: 'NoneType' object is not subscriptable

In [ ]:
# 取消关注-删

# to be done
# 从前端获取哪个用户取消关注了哪个公众号
user_name = 'new_user_1'
channel_name = 'ScienceDaily'

#要删除subscribe表的内容
user_id = 0
channel_id = 0

# 调试状态
debugg = 1

with connection.cursor(cursor=pymysql.cursors.DictCursor) as cursor:
    #获取uid
    sql_get_user_id_from_username = """
    SELECT users.uid
    FROM users
    WHERE users.username = %s
    """
    cursor.execute(sql_get_user_id_from_username,(user_name))
    result = cursor.fetchone()
    user_id = result['uid']
    
    #获取nid
    sql_get_channel_id_from_channel_name = """
    SELECT channels.nid
    FROM channels
    WHERE channels.name = %s
    """
    cursor.execute(sql_get_channel_id_from_channel_name,(channel_name))
    result = cursor.fetchone()
    channel_id = result['nid']
    
    #将subscribe表中(uid,nid)删除
    sql_unsubscribe = """
        DELETE FROM subscribe
        WHERE subscribe.uid = %s AND subscribe.nid = %s
    """
    cursor.execute(sql_unsubscribe,(user_id,channel_id))
    
    # to be done
    ##### 通知前端取消关注成功
    
    #####
    
    #debug
    if debugg:
        sql_debug = """
            SELECT *
            FROM subscribe AS s
            WHERE s.uid = %s AND s.nid = %s
        """
        cursor.execute(sql_debug,(user_id,channel_id))
        result = cursor.fetchall()
        if len(result) == 0:
            print('成功取关')
        else:
            for row in result:
                print(row)


成功取关


In [ ]:
# （当前用户）显示自己所关注的公众号的全部推文-查

# to be done
# 从前端获取哪个用户要显示自己所关注的公众号的全部推文
user_name = 'new_user_1'

#根据user_name找uid
user_id = 0

# 调试状态
debugg = 1

with connection.cursor(cursor=pymysql.cursors.DictCursor) as cursor:
    #获取uid
    sql_get_user_id_from_username = """
    SELECT users.uid
    FROM users
    WHERE users.username = %s
    """
    cursor.execute(sql_get_user_id_from_username,(user_name))
    result = cursor.fetchone()
    user_id = result['uid']
    
    #输出uid用户关注的公众号的全部推文
    sql_post_content_subscribed_by_uid = """
        SELECT posts.content
        FROM subscribe JOIN posts
        WHERE subscribe.uid = %s AND subscribe.nid = posts.from_channel
    """
    cursor.execute(sql_post_content_subscribed_by_uid,(user_id))
    
    # debug
    if debugg:
        result = cursor.fetchall()
        for row in result:
            print(row)
    
    # to be done
    ##### 把推文的content给前端
    
    #####


{'content': 'Recent developments in space research'}


In [ ]:
# （当前用户）评论某推文-增

# to be done
# 从前端获取哪个用户要评论哪个推文(以标题形式告知),内容是什么
user_name = 'new_user_1'
posts_title = 'AI Breakthroughs'
comment_content = 'This is a test comment text for debug'

#根据user_name找uid,posts_title找pid
user_id = 0
posts_id = 0

# 调试状态
debugg = 1

with connection.cursor(cursor=pymysql.cursors.DictCursor) as cursor:
    #获取uid
    sql_get_user_id_from_username = """
    SELECT users.uid
    FROM users
    WHERE users.username = %s
    """
    cursor.execute(sql_get_user_id_from_username,(user_name))
    result = cursor.fetchone()
    user_id = result['uid']
    
    #获取pid
    sql_get_post_id_from_post_title = """
    SELECT posts.pid
    FROM posts
    WHERE posts.title = %s
    """
    cursor.execute(sql_get_post_id_from_post_title,(posts_title))
    result = cursor.fetchone()
    posts_id = result['pid']
    
    #获取pid的评论树中编号最大的那一个结点的编号 max_cid
    sql_get_max_cid_of_pid = """
    SELECT max(cid)
    FROM comments
    GROUP BY pid
    HAVING pid = %s
    """
    cursor.execute(sql_get_max_cid_of_pid,(posts_id))
    result = cursor.fetchone()
    max_cid = result['max(cid)'] #编号最大的那一个结点的编号
    
    #向数据库中增加用户uid对推文pid的评论comment_content
    sql_add_comment_from_uid_to_pid = """
    INSERT INTO comments (cid, pid, from_user, to_comment, master_comment, content)
    VALUES(%s,%s,%s,NULL,NULL,%s)
    """
    cursor.execute(sql_add_comment_from_uid_to_pid,(max_cid + 1,posts_id,user_id,comment_content))

In [ ]:
# （当前用户）评论某评论-增

# to be done
# 从前端获取哪个用户要评论哪个推文(以标题形式告知)下的哪一条评论(以cid告知,因为内容不UNIQUE)
# 从前端获取用户发表的评论内容(comment_content)
user_name = 'new_user_1'
posts_title = 'AI Breakthroughs'
target_comment_id = 4
comment_content = 'This is a test comment text for commenting another comment'

#根据user_name找uid,根据posts_title找pid
user_id = 0
posts_id = 0
# 调试状态
debugg = 1

with connection.cursor(cursor=pymysql.cursors.DictCursor) as cursor:
    #获取uid
    sql_get_user_id_from_username = """
    SELECT users.uid
    FROM users
    WHERE users.username = %s
    """
    cursor.execute(sql_get_user_id_from_username,(user_name))
    result = cursor.fetchone()
    user_id = result['uid']
    
    #获取pid
    sql_get_post_id_from_post_title = """
    SELECT posts.pid
    FROM posts
    WHERE posts.title = %s
    """
    cursor.execute(sql_get_post_id_from_post_title,(posts_title))
    result = cursor.fetchone()
    posts_id = result['pid']
    
    #获取pid的评论树中编号最大的那一个结点的编号 max_cid
    sql_get_max_cid_of_pid = """
    SELECT max(cid)
    FROM comments
    GROUP BY pid
    HAVING pid = %s
    """
    cursor.execute(sql_get_max_cid_of_pid,(posts_id))
    result = cursor.fetchone()
    max_cid = result['max(cid)'] #编号最大的那一个结点的编号
    
    #获取pid的评论树中target_comment_id评论对应的根评论的cid
    #如果target_comment_id评论本身就是根评论,则其
    sql_get_master_comment_id="""
    SELECT *
    FROM comments
    WHERE comments.pid = %s AND comments.cid = %s
    """
    cursor.execute(sql_get_master_comment_id,(posts_id,target_comment_id))
    result = cursor.fetchone()
    if result['master_comment'] is not None:
        master_comment_id = result['master_comment'] #根评论的cid
    else:
        master_comment_id = target_comment_id
    
    #向数据库中增加用户uid对推文pid的评论target_comment_id的评论comment_content
    sql_add_comment_from_uid_to_pid_on_target_comment_id = """
    INSERT INTO comments (cid, pid, from_user, to_comment, master_comment, content)
    VALUES(%s,%s,%s,%s,%s,%s)
    """
    cursor.execute(sql_add_comment_from_uid_to_pid_on_target_comment_id,
                   (max_cid + 1,posts_id,user_id,target_comment_id,master_comment_id,comment_content))
    
    #debug 展示帖子posts_id下的所有评论
    if debugg:
        sql_show_comments_under_posts_id="""
        SELECT *
        FROM comments
        WHERE comments.pid = %s
        """
        cursor.execute(sql_show_comments_under_posts_id,(posts_id))
        result = cursor.fetchall()
        for row in result:
            print(row)

{'cid': 0, 'pid': 1, 'from_user': 2, 'to_comment': None, 'master_comment': None, 'content': 'Great insights on AI!', 'created_at': datetime.datetime(2025, 4, 10, 15, 40, 57)}
{'cid': 1, 'pid': 1, 'from_user': 3, 'to_comment': 0, 'master_comment': 0, 'content': 'I agree, especially about neural networks.', 'created_at': datetime.datetime(2025, 4, 10, 15, 40, 57)}
{'cid': 2, 'pid': 1, 'from_user': 5, 'to_comment': 0, 'master_comment': 0, 'content': 'What about ethical considerations?', 'created_at': datetime.datetime(2025, 4, 10, 15, 40, 57)}
{'cid': 3, 'pid': 1, 'from_user': 1, 'to_comment': 1, 'master_comment': 0, 'content': 'Thanks! I think ethics is crucial too.', 'created_at': datetime.datetime(2025, 4, 10, 15, 40, 57)}
{'cid': 4, 'pid': 1, 'from_user': 4, 'to_comment': 2, 'master_comment': 0, 'content': "That's a good point. We need more discussion on AI ethics.", 'created_at': datetime.datetime(2025, 4, 10, 15, 40, 57)}
{'cid': 5, 'pid': 1, 'from_user': 6, 'to_comment': None, 'mas

In [ ]:
# （当前用户）点赞某推文-增

# to be done
# 从前端获取哪个用户要点赞哪个推文(以标题形式告知)
user_name = 'new_user_1'
posts_title = 'AI Breakthroughs'

#根据user_name找uid,posts_title找pid
user_id = 0
posts_id = 0

# 调试状态
debugg = 1

with connection.cursor(cursor=pymysql.cursors.DictCursor) as cursor:
    #获取uid
    sql_get_user_id_from_username = """
    SELECT users.uid
    FROM users
    WHERE users.username = %s
    """
    cursor.execute(sql_get_user_id_from_username,(user_name))
    result = cursor.fetchone()
    user_id = result['uid']
    
    #获取pid
    sql_get_post_id_from_post_title = """
    SELECT posts.pid
    FROM posts
    WHERE posts.title = %s
    """
    cursor.execute(sql_get_post_id_from_post_title,(posts_title))
    result = cursor.fetchone()
    posts_id = result['pid']
    
    #向数据库中增加用户uid对推文pid的点赞
    #如果已经点过赞就不用添加了
    sql_has_already_liked_posts="""
    SELECT *
    FROM like_comment
    WHERE like_comment.uid = %s AND like_comment.pid = %s
    """
    cursor.execute(sql_has_already_liked_posts,(user_id,posts_id))
    result = cursor.fetchone()
    if result is None:
        sql_add_like_comment = """
        INSERT INTO like_comment (uid, pid)
        VALUES(%s,%s)
        """
        cursor.execute(sql_add_like_comment,(user_id,posts_id))
    else:
        print('已经赞过了')

In [ ]:
# （当前用户）点赞某评论-增

# to be done
# 从前端获取哪个用户要点赞哪个推文(以标题形式告知)的哪条评论(以cid告知)
user_name = 'new_user_2'
posts_title = 'AI Breakthroughs'
target_comment_id = 4

#根据user_name找uid,posts_title找pid
user_id = 0
posts_id = 0

# 调试状态
debugg = 1

with connection.cursor(cursor=pymysql.cursors.DictCursor) as cursor:
    #获取uid
    sql_get_user_id_from_username = """
    SELECT users.uid
    FROM users
    WHERE users.username = %s
    """
    cursor.execute(sql_get_user_id_from_username,(user_name))
    result = cursor.fetchone()
    user_id = result['uid']
    
    #获取pid
    sql_get_post_id_from_post_title = """
    SELECT posts.pid
    FROM posts
    WHERE posts.title = %s
    """
    cursor.execute(sql_get_post_id_from_post_title,(posts_title))
    result = cursor.fetchone()
    posts_id = result['pid']
    
    #向数据库中增加用户uid对推文pid的评论target_comment_id点赞
    #如果已经点过赞就不用添加了
    sql_has_already_liked_comment="""
    SELECT *
    FROM like_comment
    WHERE like_comment.uid = %s AND like_comment.pid = %s AND like_comment.cid = %s
    """
    cursor.execute(sql_has_already_liked_comment,(user_id,posts_id,target_comment_id))
    result = cursor.fetchone()
    if result is None:
        sql_add_like_comment = """
        INSERT INTO like_comment (uid, cid, pid)
        VALUES(%s,%s,%s)
        """
        cursor.execute(sql_add_like_comment,(user_id,target_comment_id,posts_id))
    else:
        print('已经赞过了')

In [ ]:
# （当前用户）取消点赞某推文-删

# to be done
# 从前端获取哪个用户要点赞哪个推文(以标题形式告知)
user_name = 'new_user_1'
posts_title = 'AI Breakthroughs'

#根据user_name找uid,posts_title找pid
user_id = 0
posts_id = 0

# 调试状态
debugg = 1

with connection.cursor(cursor=pymysql.cursors.DictCursor) as cursor:
    #获取uid
    sql_get_user_id_from_username = """
    SELECT users.uid
    FROM users
    WHERE users.username = %s
    """
    cursor.execute(sql_get_user_id_from_username,(user_name))
    result = cursor.fetchone()
    user_id = result['uid']
    
    #获取pid
    sql_get_post_id_from_post_title = """
    SELECT posts.pid
    FROM posts
    WHERE posts.title = %s
    """
    cursor.execute(sql_get_post_id_from_post_title,(posts_title))
    result = cursor.fetchone()
    posts_id = result['pid']
    
    #从数据库中删除用户uid对推文pid的点赞
    sql_delete_like_post = """
    DELETE FROM like_post
    WHERE like_post.uid = %s AND like_post.pid = %s
    """
    cursor.execute(sql_delete_like_post,(user_id,posts_id))

In [ ]:
# （当前用户）取消点赞某评论-删

# to be done
# 从前端获取哪个用户要点赞哪个推文(以标题形式告知)的哪条评论(以cid告知)
user_name = 'new_user_2'
posts_title = 'AI Breakthroughs'
target_comment_id = 4

#根据user_name找uid,posts_title找pid
user_id = 0
posts_id = 0

# 调试状态
debugg = 1

with connection.cursor(cursor=pymysql.cursors.DictCursor) as cursor:
    #获取uid
    sql_get_user_id_from_username = """
    SELECT users.uid
    FROM users
    WHERE users.username = %s
    """
    cursor.execute(sql_get_user_id_from_username,(user_name))
    result = cursor.fetchone()
    user_id = result['uid']
    
    #获取pid
    sql_get_post_id_from_post_title = """
    SELECT posts.pid
    FROM posts
    WHERE posts.title = %s
    """
    cursor.execute(sql_get_post_id_from_post_title,(posts_title))
    result = cursor.fetchone()
    posts_id = result['pid']
    
    #从数据库中删除用户uid对推文pid的点赞
    sql_delete_like_comment = """
    DELETE FROM like_comment
    WHERE like_comment.uid = %s AND like_comment.pid = %s AND like_comment.cid = %s
    """
    cursor.execute(sql_delete_like_comment,(user_id,posts_id,target_comment_id))

In [ ]:
#用于调试
with connection.cursor(cursor=pymysql.cursors.DictCursor) as cursor:
    cursor.execute("""
                SELECT * 
                FROM like_comment
                """)
    result = cursor.fetchall()
    for row in result:
        print(row)

{'uid': 1, 'cid': 0, 'pid': 1}
{'uid': 1, 'cid': 3, 'pid': 1}
{'uid': 2, 'cid': 1, 'pid': 2}
{'uid': 2, 'cid': 4, 'pid': 2}
{'uid': 3, 'cid': 0, 'pid': 5}
{'uid': 3, 'cid': 2, 'pid': 5}
{'uid': 4, 'cid': 4, 'pid': 5}
{'uid': 5, 'cid': 0, 'pid': 1}
{'uid': 5, 'cid': 1, 'pid': 1}
{'uid': 5, 'cid': 2, 'pid': 1}
{'uid': 5, 'cid': 3, 'pid': 1}
{'uid': 5, 'cid': 4, 'pid': 1}
{'uid': 5, 'cid': 0, 'pid': 2}
{'uid': 5, 'cid': 1, 'pid': 2}
{'uid': 5, 'cid': 2, 'pid': 2}
{'uid': 5, 'cid': 3, 'pid': 2}
{'uid': 5, 'cid': 4, 'pid': 2}
{'uid': 5, 'cid': 0, 'pid': 5}
{'uid': 5, 'cid': 1, 'pid': 5}
{'uid': 5, 'cid': 2, 'pid': 5}
{'uid': 5, 'cid': 3, 'pid': 5}
{'uid': 5, 'cid': 4, 'pid': 5}
{'uid': 7, 'cid': 4, 'pid': 1}
